In [1]:
#import findspark
#findspark.init()
from pyspark import SparkContext,SparkConf

def create_sc(pyFiles):
    sc_conf = SparkConf()
    sc_conf.setAppName("Weather_PCA")
    sc_conf.set('spark.executor.memory', '2g')
    sc_conf.set('spark.executor.cores', '4')
    sc_conf.set('spark.cores.max', '40')
    sc_conf.set('spark.default.parallelism','10')
    sc_conf.set('spark.logConf', True)
    print(sc_conf.getAll())

    sc = SparkContext(conf=sc_conf,pyFiles=pyFiles)

    return sc 

#sc.stop()
#sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])
sc = create_sc(pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStatistics.py'])

dict_items([('spark.app.name', 'Weather_PCA'), ('spark.executor.memory', '2g'), ('spark.executor.cores', '4'), ('spark.cores.max', '40'), ('spark.default.parallelism', '10'), ('spark.logConf', 'True')])


In [2]:
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import numpy as np
from lib.computeStatistics import *

In [4]:
!aws s3 cp --recursive --quiet  s3://dse-weather/NY.parquet /mnt/workspace/Data/NY.parquet

In [5]:
!du /mnt/workspace/Data/

424	/mnt/workspace/Data/US_stations.parquet/US_stations.parquet
848	/mnt/workspace/Data/US_stations.parquet
990536	/mnt/workspace/Data/US_weather.parquet/US_weather.parquet
1981072	/mnt/workspace/Data/US_weather.parquet
28244	/mnt/workspace/Data/NY.parquet
2010164	/mnt/workspace/Data/


In [6]:
!hdfs dfs -mkdir weather
!hdfs dfs -ls

Found 2 items
drwxr-xr-x   - hadoop hadoop          0 2018-04-01 17:49 .sparkStaging
drwxr-xr-x   - hadoop hadoop          0 2018-04-01 17:50 weather


In [7]:
!hdfs dfs -copyFromLocal /mnt/workspace/Data/NY.parquet /weather/NY.parquet

In [8]:
!hdfs dfs -du /weather/

28885752    /weather/NY.parquet
847798      /weather/US_stations.parquet
2027712578  /weather/US_weather.parquet


In [9]:
%%time
df=sqlContext.read.parquet('/weather/NY.parquet')
print(df.count())
df.show(5)

84199
+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
|USC00303452|       PRCP|1907|[00 00 00 00 60 5...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 5 rows

CPU times: user 0 ns, sys: 8 ms, total: 8 ms
Wall time: 13.3 s


In [10]:
from time import time
t=time()

N=sc.defaultParallelism
print('Number of executors=',N)
print('took',time()-t,'seconds')

Number of executors= 10
took 0.0008840560913085938 seconds


In [11]:
data_dir='/mnt/workspace/Data/'


In [12]:
%%time
from pickle import dump

state='NY'
parquet=state+'.parquet'
df=sqlContext.read.parquet('/weather/'+parquet).cache()
print(state,df.count())

NY 84199
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.17 s


In [13]:
%%time
df.show(4)

+-----------+-----------+----+--------------------+-----+
|    Station|Measurement|Year|              Values|State|
+-----------+-----------+----+--------------------+-----+
|USC00303452|       PRCP|1903|[00 7E 00 7E 00 7...|   NY|
|USC00303452|       PRCP|1904|[00 00 28 5B 00 0...|   NY|
|USC00303452|       PRCP|1905|[00 00 60 56 60 5...|   NY|
|USC00303452|       PRCP|1906|[00 00 00 00 00 0...|   NY|
+-----------+-----------+----+--------------------+-----+
only showing top 4 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 113 ms


In [14]:
# %load lib/computeStatistics.py

from numpy import linalg as LA
import numpy as np

from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from time import time

def computeStatistics(sqlContext,df):
    """Compute all of the statistics for a given dataframe
    Input: sqlContext: to perform SQL queries
            df: dataframe with the fields 
            Station(string), Measurement(string), Year(integer), Values (byteArray with 365 float16 numbers)
    returns: STAT, a dictionary of dictionaries. First key is measurement, 
             second keys described in computeStats.STAT_Descriptions
    """

    sqlContext.registerDataFrameAsTable(df,'weather')
    STAT={}  # dictionary storing the statistics for each measurement
    measurements=['TMAX', 'SNOW', 'SNWD', 'TMIN', 'PRCP', 'TOBS']
    
    for meas in measurements:
        t=time()
        Query="SELECT * FROM weather\n\tWHERE measurement = '%s'"%(meas)
        mdf = sqlContext.sql(Query)
        print(meas,': shape of mdf is ',mdf.count())

        data=mdf.rdd.map(lambda row: unpackArray(row['Values'],np.float16))

        #Compute basic statistics
        STAT[meas]=computeOverAllDist(data)   # Compute the statistics 

        # compute covariance matrix
        OUT=computeCov(data)

        #find PCA decomposition
        eigval,eigvec=LA.eig(OUT['Cov'])

        # collect all of the statistics in STAT[meas]
        STAT[meas]['eigval']=eigval
        STAT[meas]['eigvec']=eigvec
        STAT[meas].update(OUT)

        print('time for',meas,'is',time()-t)
    
    return STAT

# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
    L=int(len(SortedVals)/percentile)
    return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
    UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
    flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
    count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
    mean=S1/count
    std=np.sqrt(S2/count-mean**2)
    Vals=flat.sample(False,0.0001).collect()
    SortedVals=np.array(sorted(Vals))
    low100,high100=find_percentiles(SortedVals,100)
    low1000,high1000=find_percentiles(SortedVals,1000)
    return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

# description of data returned by computeOverAllDist
STAT_Descriptions=[
('SortedVals', 'Sample of values', 'vector whose length varies between measurements'),
 ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'),
 ('mean', 'mean value', ()),
 ('std', 'std', ()),
 ('low100', 'bottom 1%', ()),
 ('high100', 'top 1%', ()),
 ('low1000', 'bottom 0.1%', ()),
 ('high1000', 'top 0.1%', ()),
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
]




In [15]:
!echo \$PYSPARK_PYTHON, \$PYSPARK_DRIVER_PYTHON

$PYSPARK_PYTHON, $PYSPARK_DRIVER_PYTHON


In [29]:
import os
os.environ['PYSPARK_PYTHON']='/mnt/anaconda/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON']='/mnt/anaconda/bin/python3'
# PYSPARK_PYTHON, PYSPARK_DRIVER_PYTHON
Environ_vars=['SPARK_HOME','PYSPARK_PYTHON', 'PYSPARK_DRIVER_PYTHON']
for var in Environ_vars:
    print(var,var in os.environ)


SPARK_HOME True
PYSPARK_PYTHON True
PYSPARK_DRIVER_PYTHON True


### Compatibility error

The relevant part of the error messages is:
```
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.
```

If I try to check the variables using the terminal:
```sh
echo $PYSPARK_PYTHON, $PYSPARK_DRIVER_PYTHON
```

I get:
```
,
```

In [30]:
%%time
STAT=computeStatistics(sqlContext,df)

TMAX : shape of mdf is  13437


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 13.0 failed 4 times, most recent failure: Lost task 2.3 in stage 13.0 (TID 95, ip-10-129-224-190.ec2.internal, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/mnt/yarn/usercache/hadoop/appcache/application_1522599867271_0002/container_1522599867271_0002_01_000002/pyspark.zip/pyspark/worker.py", line 123, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.6, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:234)
	at org.apac

In [28]:
filename=data_dir+'STAT_%s.pickle'%state
!ls -l $data_dir
dump((STAT,STAT_Descriptions),open(filename,'wb'))

total 32
drwxrwxr-x 2 hadoop hadoop  4096 Apr  1 17:49 NY.parquet
drwxrwxr-x 3 hadoop hadoop  4096 Apr  1 16:23 US_stations.parquet
drwxrwxr-x 3 hadoop hadoop 20480 Apr  1 16:23 US_weather.parquet


NameError: name 'STAT' is not defined

In [9]:
X=STAT['TMAX']['Var']
for key in STAT.keys():
    Y=STAT[key]['Var']
    print(key,sum(abs(X-Y)))

TMAX 0.0
SNOW 852107.705867
SNWD 4464167.85212
TMIN 319734.5315
PRCP 1184305.12284
TOBS 277719.008938


In [11]:
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  25684434 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle
-rw-r--r--  1 yoavfreund  staff  17545930 Mar 18 17:38 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  26741490 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [12]:
!gzip -f ../../Data/Weather/STAT*.pickle
!ls -l ../../Data/Weather/STAT*

-rw-r--r--  1 yoavfreund  staff  14948048 Mar 18 20:30 ../../Data/Weather/STAT_NY.pickle.gz
-rw-r--r--  1 yoavfreund  staff  11717259 Mar 18 20:19 ../../Data/Weather/STAT_RI.pickle.gz
-rw-r--r--  1 yoavfreund  staff  13522496 Mar 10 12:06 ../../Data/Weather/STAT_SSSSBBBB.pickle.gz


In [13]:
for state in ['RI','NY']:
    command="aws s3  cp ../../Data/Weather/STAT_%s.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_%s.pickle.gz"%(state,state)
    print(command)
    !$command

aws s3  cp ../../Data/Weather/STAT_RI.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
upload: ../../Data/Weather/STAT_RI.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_RI.pickle.gz
aws s3  cp ../../Data/Weather/STAT_NY.pickle.gz s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz
upload: ../../Data/Weather/STAT_NY.pickle.gz to s3://mas-dse-open/Weather/by_state/STAT_NY.pickle.gz


In [14]:
!aws s3  ls s3://mas-dse-open/Weather/by_state/ | grep STAT

2018-03-18 20:34:12   14948048 STAT_NY.pickle.gz
2018-03-18 20:33:54   11717259 STAT_RI.pickle.gz
